In [1]:
cd '../..'

/home/tyz910/src/sberbank-covid19


In [2]:
import re
import json
import requests
import pandas as pd
from datetime import datetime

countries = pd.read_csv('data/countries.csv')
city_map = {row['region_center']: row['country'] for i, row in countries[countries['is_region'] == 1].iterrows()}

body = requests.get('https://yandex.ru/web-maps/covid19/isolation').content
data = json.loads(re.compile(r'class="config-view">(.+?)<').search(body.decode('utf-8'))[1])

In [3]:
def ts_to_date(ts):
    return datetime.utcfromtimestamp(ts + 3 * 60 * 60).strftime('%Y-%m-%d')


result = []

for c in data['covidData']['cities']:
    if c['name'] in city_map:
        country = city_map[c['name']]
        result.append(
            pd.DataFrame(
                [[ts_to_date(r['ts']), country, r['value']] for r in c['histogramDays']], 
                columns=['date', 'country', 'isolation'],
            )
        )

result = pd.concat(result).reset_index(drop=True)
result.to_csv('data/mobility-yandex.csv', index=False)
result.head()

,date,country,isolation
0,2020-02-23,RU-SPE,2.5
1,2020-02-24,RU-SPE,2.6
2,2020-02-25,RU-SPE,0.4
3,2020-02-26,RU-SPE,0.4
4,2020-02-27,RU-SPE,0.4
